In [1]:
import numpy as np
import matplotlib.pylab as plt
import corner
import numpy as np
import glob
from PTMCMCSampler import PTMCMCSampler
from PTMCMCSampler import proposals

%matplotlib inline

Do not have mpi4py package.
Do not have emcee package


## List of available jump proposals

`PTMCMC` already has a list of default jump proposals. These can be viewed by running

In [2]:
proposals.available_jump_proposals()

Available jump proposals:

SingleComponentAdaptiveCovariance
AdaptiveCovariance
SingleComponentAdaptiveGaussian
MultiComponentAdaptiveGaussian
AdaptiveGaussian
DifferentialEvolution
Normal
Uniform
Prior


## Default jump proposals

By default the following jump proposals will be used

In [42]:
proposals.default_jump_proposals()

Default jump proposals:

SingleComponentAdaptiveCovariance
AdaptiveCovariance
SingleComponentAdaptiveGaussian
MultiComponentAdaptiveGaussian
AdaptiveGaussian
DifferentialEvolution


Each jump proposal will have an equal weighting. This means that each jump proposal has equal probability of being picked.

## Choosing specific jump proposals

Of course, having equal weighting for all jump proposals might not suit your problem. You are able to specify different weights for different jump proposals by passing a dictionary called weights to the `sampler` object. This dictionary might have the following form:

In [43]:
weights = {"SingleComponentAdaptiveCovariance": 5,
           "AdaptiveCovariance": 1,
           "SingleComponentAdaptiveGaussian": 1,
           "MultiComponentAdaptiveGaussian": 1,
           "AdaptiveGaussian": 1,
           "DifferentialEvolution": 1}

Here, the `SingleComponentAdaptiveCovariance` jump proposal will have a probaility of being used equal to `5/10 = 1/2` while the other jump proposals have probability of being chosen equal to `1/10`.

If you only want to use the `SingleComponentAdaptiveCovariance` jump proposal, then this can be done by writing your weight dictionary as:

In [44]:
weights = {"SingleComponentAdaptiveCovariance": 5}

Some jump proposals (like `Normal` and `Uniform`) require addtional keyword arguments. These keyword arguments are passed to the `sampler` object by using the `jump_proposal_arguments` keyword argument. For example, if you want to use the `Uniform` jump proposal with bounds of `0` and `100` you would define a jump_proposal_argument dictionary as:

In [45]:
jump_proposal_arguments = {"Uniform": {"pmin": 0, "pmax": 100}}

## Custom jump proposals

Custom jump proposals can be added very easily. Lets say that you would like to add the following jump proposal

In [3]:
class CustomJump(object):
    """Class to handle my custom jump proposal
    """
    def __init__(self, pmin, mode, pmax):
        self.pmin = pmin
        self.mode = mode
        self.pmax = pmax
        
    def jump(self, samples):
        new_samples = np.random.triangular(self.pmin, self.mode, self.pmax, len(samples))
        return new_samples, 0.0

You can then add this by running:

In [32]:
customjump = CustomJump(0, 5, 8)
sampler.addProposalToCycle(customjump.jump, 10)

Where we have chosen to give our customjump proposal a weighting of 10

## Example

In [4]:
class GaussianLikelihood(object):
    
    def __init__(self, ndim=2, pmin=-10, pmax=10):
        
        self.a = np.ones(ndim)*pmin
        self.b = np.ones(ndim)*pmax
        
        # get means
        self.mu = np.random.uniform(pmin, pmax, ndim)

        # ... and a positive definite, non-trivial covariance matrix.
        cov  = 0.5-np.random.rand(ndim**2).reshape((ndim, ndim))
        cov  = np.triu(cov)
        cov += cov.T - np.diag(cov.diagonal())
        self.cov  = np.dot(cov,cov)

        # Invert the covariance matrix first.
        self.icov = np.linalg.inv(self.cov)
        
    def lnlikefn(self, x):
        diff = x - self.mu
        return -np.dot(diff,np.dot(self.icov, diff))/2.0
    
    def lnpriorfn(self, x):
        
        if np.all(self.a <= x) and np.all(self.b >= x):
            return 0.0
        else:
            return -np.inf      

In [5]:
ndim = 2
pmin, pmax = 0.0, 10.0
glo = GaussianLikelihood(ndim=ndim, pmin=5., pmax=6.)

In [6]:
p0 = np.random.uniform(pmin, pmax, ndim)
cov = np.eye(ndim) * 0.1**2

In [7]:
sampler = PTMCMCSampler.PTSampler(ndim, glo.lnlikefn, glo.lnpriorfn, np.copy(cov), outDir='./chains')

In [8]:
weights = {"Uniform": 1}

jump_proposal_arguments = {"Uniform": {"pmin": 0, "pmax": 10}}

In [9]:
customjump = CustomJump(0, 5, 8)
sampler.addProposalToCycle(customjump.jump, 10)

In [10]:
sampler.sample(p0, 100000, weights=weights, jump_proposal_arguments=jump_proposal_arguments,
               burn=500, thin=1, covUpdate=500)

Finished 99.00 percent in 15.476101 s Acceptance rate = 0.0284444
Run Complete


array([[5.5024261 , 5.48829797],
       [5.5024261 , 5.48829797],
       [5.5024261 , 5.48829797],
       ...,
       [5.24837589, 5.13255974],
       [5.39158873, 5.85375395],
       [5.39158873, 5.85375395]])